In [11]:
# !pip install pandas transformers torch numpy matplotlib evaluate datasets
# !pip install torch
!pip install transformers[torch]
!pip install accelerate -U

zsh:1: no matches found: transformers[torch]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import numpy as np
import evaluate
from huggingface_hub import login
from datasets import load_dataset, Dataset

/Users/abhiraj/Espresso/Developer_Stuff/projects/email-spam-detection-nlp/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load data

In [2]:
dataset = load_dataset("legacy107/spamming-email-classification")

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Text', 'Spam'],
        num_rows: 4556
    })
    val: Dataset({
        features: ['Text', 'Spam'],
        num_rows: 569
    })
    test: Dataset({
        features: ['Text', 'Spam'],
        num_rows: 570
    })
})

In [4]:
train_data = dataset['train'].to_pandas()
test_data = dataset['test'].to_pandas()

In [5]:
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

In [6]:
# Tokenizer
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

def tokenize_function(examples):
    return tokenizer(examples['Text'], padding="max_length", truncation=True)

# Tokenize the datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 570/570 [00:00<00:00, 4130.41 examples/s]


In [7]:
# Model
model = AutoModelForSequenceClassification.from_pretrained('sentence-transformers/all-MiniLM-L6-v2', num_labels=2)

# Evaluation metric
metric = evaluate.load("accuracy")

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return metric.compute(predictions=preds, references=p.label_ids)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
# Training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    max_steps=50,
    learning_rate=2e-4,
    logging_steps=1,
    output_dir="outputs",
    optim="adamw_hf",
    fp16=False  # Ensure mixed precision is not used
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

ValueError: fp16 mixed precision requires a GPU (not 'mps').